In [11]:
## install libs
import urllib.parse
from functools import cache
from typing import Any

import earthaccess
import requests
import requests.cookies
import pandas as pd
import re

In [3]:
# handle auth headers being deleted on redirects outside urs (auth.py needs a fix)
class BasicAuthResponseHook:
    def __init__(self, auth: earthaccess.Auth) -> None:
        self.auth = auth

    def __call__(self, r: requests.Response, **kwargs: Any) -> requests.Response:
        if urllib.parse.urlparse(r.url).hostname != self.auth.system.edl_hostname:
            return r

        # Consume content and release the original connection to allow our new request
        # to reuse the same one.
        r.content
        r.close()

        prepared_request = r.request.copy()
        requests.cookies.extract_cookies_to_jar(
            prepared_request._cookies, r.request, r.raw  # type: ignore
        )
        prepared_request.prepare_cookies(prepared_request._cookies)  # type: ignore
        prepared_request.prepare_auth((self.auth.username, self.auth.password))

        _r = r.connection.send(prepared_request, **kwargs)
        _r.history.append(r)
        _r.request = prepared_request

        return _r

In [4]:
# fetch all datasets
results = earthaccess.search_data(
    provider="SEDAC",
)

In [6]:
# auth
auth = earthaccess.login()
earthaccess.__store__.auth.get_session = cache(earthaccess.__store__.auth.get_session)
session = earthaccess.get_requests_https_session()
session.hooks["response"].append(BasicAuthResponseHook(auth))

In [8]:
results

[Collection: {'ShortName': 'CIESIN_SEDAC_USPAT_HUP', 'Version': '1.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180.0, 'EastBoundingCoordinate': 180.0, 'NorthBoundingCoordinate': 85.0, 'SouthBoundingCoordinate': -60.0}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '-3700-01-01T00:00:00.000Z', 'EndingDateTime': '-2000-12-31T00:00:00.000Z'}}
 Size(MB): 0.83
 Data: ['https://sedac.ciesin.columbia.edu/downloads/data/urbanspatial/urbanspatial-hist-urban-pop-3700bc-ad2000/urbanspatial-hist-urban-pop-3700bc-ad2000-xlsx.xlsx'],
 Collection: {'ShortName': 'CIESIN_SEDAC_ANTHROMES_v2_1700', 'Version': '2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180.0, 'EastBoundingCoordinate': 180.0, 'NorthBoundingCoordinate': 90.0, 'SouthBoundingCoordinate': -90.0}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '1700-01-01T00:00:00.00

In [ ]:
filenames = earthaccess.download(granules=results)
print(filenames)